# Srock Strategy - Factor Strategy 0003 - Factor-Based Stock Selection 

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [2]:
sd = '2019-12-31'
ed = '2026-02-13'

In [3]:
sql_factor =  f"""
SELECT 
    date, 
    instrument,
    float_market_cap AS factor,
FROM cn_stock_prefactors
WHERE is_risk_warning = 0
AND list_sector IN (1, 2, 3)
AND list_days > 365
ORDER BY date, factor
"""

In [4]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 5
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [5]:
df_strategy = dai.query(sql_trade, filters = {"date":[sd, ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2019-12-31,300028.SZ,2.648608e+08,1.0,0.2
1,2019-12-31,603922.SH,4.928000e+08,2.0,0.2
2,2019-12-31,603709.SH,5.040000e+08,3.0,0.2
3,2019-12-31,603331.SH,5.201475e+08,4.0,0.2
4,2019-12-31,603238.SH,5.211000e+08,5.0,0.2
...,...,...,...,...,...
7425,2026-02-13,603307.SH,7.056700e+08,1.0,0.2
7426,2026-02-13,001260.SZ,7.251700e+08,2.0,0.2
7427,2026-02-13,301287.SZ,7.752671e+08,3.0,0.2
7428,2026-02-13,301309.SZ,7.826294e+08,4.0,0.2


## 3. Backtest Strategy

In [6]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 19:24:52] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 19:24:52] [info     ] 2019-12-31, 2026-02-13, , , instruments=90
[2026-02-18 19:24:52] [info     ] bigtrader module V2.2.0
[2026-02-18 19:24:52] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10


order_target_percent: price is null for '300028.SZ' at '2019-12-31 15:00:00'
order_target_percent: price is null for '300028.SZ' at '2020-01-02 15:00:00'
order_target_percent: price is null for '300028.SZ' at '2020-01-03 15:00:00'
order_target_percent: price is null for '300028.SZ' at '2020-01-06 15:00:00'
order_target_percent: price is null for '300028.SZ' at '2020-01-07 15:00:00'
order_target_percent: price is null for '300028.SZ' at '2020-01-08 15:00:00'
order_target_percent: price is null for '300028.SZ' at '2020-01-09 15:00:00'
order_target_percent: price is null for '300028.SZ' at '2020-01-10 15:00:00'
order_target_percent: price is null for '300028.SZ' at '2020-01-13 15:00:00'
order_target_percent: price is null for '300028.SZ' at '2020-01-14 15:00:00'
order_target_percent: price is null for '300028.SZ' at '2020-01-15 15:00:00'
order_target_percent: price is null for '300028.SZ' at '2020-01-16 15:00:00'
order_target_percent: price is null for '300028.SZ' at '2020-01-17 15:00:00'

[2026-02-18 19:25:00] [info     ] bigtrader.v34 运行完成 [8.024s].
